In [ ]:
##packages used: pyradiomics==3.0.1, nibabel==5.2.1, pandas==2.0.3, SimpleITK==2.3.1, numpy==1.24.4

In [ ]:
import radiomics
from radiomics import featureextractor 
import os
import nibabel as nib
import pandas as pd
import SimpleITK as sitk
import numpy as np
import logging
from glob import glob


In [ ]:
logger = logging.getLogger("radiomics")
logger.setLevel(logging.ERROR)
logger = logging.getLogger("radiomics.glcm")
logger.setLevel(logging.ERROR)

In [ ]:
OUTPUT_FASTSURFER_DIR="./ADNI/output_fastsurfer/"
RESULTS_DIR="./results/ADNI/Radiomics/"

In [ ]:
settings = {"label": 2,"minimumROIDimensions":1,"voxelBatch":20}

extractor = radiomics.featureextractor.RadiomicsFeatureExtractor(**settings)

In [ ]:
first=os.listdir(OUTPUT_FASTSURFER_DIR)[0]
imagePath=OUTPUT_FASTSURFER_DIR+first+"/mri/norm.nii.gz"
maskPath=OUTPUT_FASTSURFER_DIR+first+"/mri/aparc.DKTatlas+aseg.deep.mgz.nii.gz"

In [ ]:
result = extractor.execute(imagePath, maskPath)

In [ ]:
feature_names=[]
for res in result:
    if res.startswith("original"):
        feature_names.append(res)

In [ ]:
for test in glob(OUTPUT_FASTSURFER_DIR+"*/", recursive = False):
    imagePath=test+"mri/norm.nii.gz"
    maskPath=test+"mri/aparc.DKTatlas+aseg.deep.mgz.nii.gz"
    if os.path.exists(imagePath) and os.path.exists(maskPath):
        filename=RESULTS_DIR+imagePath.split("/")[-3]+".csv"
        if not os.path.exists(filename):
            f = open(filename, "x")
            f.close()
            sitk_t1 = sitk.ReadImage(maskPath)
            t1 = sitk.GetArrayFromImage(sitk_t1)
            df=pd.DataFrame(columns=["path","region"]+feature_names)   
            for label in np.unique(t1):
                label=int(label)
                if not label ==0:
                    if (t1==label).sum()>1:
                        settings = {"label": label,"minimumROIDimensions":1}
                        extractor = radiomics.featureextractor.RadiomicsFeatureExtractor(**settings)
                        result = extractor.execute(imagePath, maskPath)
                        feature_names=[]
                        feature_values=[]
                        for res in result:
                            if res.startswith("original"):
                                feature_names.append(res)
                                feature_values.append(result[res].item())
                        df=pd.concat([df,pd.DataFrame([[imagePath,label]+feature_values],columns=["path","region"]+feature_names)])
            df.to_csv(filename,index=False)
    break